In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [2]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [136]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(6447, 323)

In [137]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [138]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [139]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('KIM_SI_HYEON', '김시현', '위에화')

In [140]:
%time KIM_SI_HYEON = make_produce48_df(name)

  0%|          | 0/323 [00:00<?, ?it/s]

김시현의 댓글 수: 6447
이전 댓글 모두 불러오기 시작...


 92%|█████████▏| 298/323 [29:12<02:26,  5.88s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



 93%|█████████▎| 299/323 [29:13<02:20,  5.86s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 0/6447 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 6447/6447 [1:37:18<00:00,  1.10it/s]

making dataframe done!
CPU times: user 50.8 s, sys: 4.36 s, total: 55.2 s
Wall time: 2h 8min 3s


In [141]:
KIM_SI_HYEON.shape

(6447, 3)

In [142]:
KIM_SI_HYEON.tail()

,User_ID,Comment,Date
6442,Mami Baba,응원할게요!화이팅~~~♥♥ 応援してます！頑張って~~~♥♥,2018-05-17
6443,신철우,"응원할께요,꼭데뷔하시길~.",2018-05-16
6444,수연,꼭 데뷔하세요,2018-05-15
6445,Hyeja Shin,시현이 너무 예쁜데?? 데뷔하고 오래 보고싶다!!,2018-05-14
6446,이지은,시현이 이번엔 꼭 데뷔하구 꽃길만 걷자,2018-05-14


In [143]:
pickle.dump(KIM_SI_HYEON, open('./produce48/위에화/KIM_SI_HYEON.pkl','wb'))

In [144]:
KIM_SI_HYEON_df = pickle.load(open('./produce48/위에화/KIM_SI_HYEON.pkl','rb'))

In [145]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [147]:
KIM_SI_HYEON_df.tail()

,User_ID,Comment,Date
6442,Mami Baba,응원할게요!화이팅~~~♥♥ 応援してます！頑張って~~~♥♥,2018-05-17
6443,신철우,"응원할께요,꼭데뷔하시길~.",2018-05-16
6444,수연,꼭 데뷔하세요,2018-05-15
6445,Hyeja Shin,시현이 너무 예쁜데?? 데뷔하고 오래 보고싶다!!,2018-05-14
6446,이지은,시현이 이번엔 꼭 데뷔하구 꽃길만 걷자,2018-05-14


In [148]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기